In [4]:
import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./train.npz
./.DS_Store
./.x.xml.swp
./DDC_model
./deploy.prototxt
./NumPy_tests.ipynb
./labels_train.npz
./DDC_EDA.ipynb
./faces_train.npz
./REAL_FAKE_le.pickle
./train.json
./DDC_prepare_data_to_fit.ipynb
./REAL_FAKE_le
./DDC_pytorch.ipynb
./res10_300x300_ssd_iter_140000.caffemodel
./haarcascade_frontalface_default.xml
./DDC_DLib.ipynb
./DDC_model.ipynb
./DDC.model
./DDC_eval.ipynb
./DDC_cv2_CascadeClassifier.ipynb
./DDC_cv2_dnn_res10_300x300_ssd_iter_140000.ipynb
./saved_models/DDC_ResNet29v2_model.006.h5
./saved_models/DDC_ResNet29v2_model.012.h5
./saved_models/DDC_ResNet29v2_model.002.h5
./saved_models/DDC_ResNet29v2_model.028.h5
./saved_models/DDC_ResNet29v2_model.009.h5
./saved_models/DDC_ResNet29v2_model.029.h5
./saved_models/DDC_ResNet29v2_model.004.h5
./saved_models/DDC_ResNet29v2_model.001.h5
./.ipynb_checkpoints/DDC_cv2_dnn_res10_300x300_ssd_iter_140000-checkpoint.ipynb
./.ipynb_checkpoints/DDC_eval-checkpoint.ipynb
./.ipynb_checkpoints/NumPy_tests-checkpoint.ipynb
./.ipynb

In [5]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt 
from keras.models import load_model

Using TensorFlow backend.


In [6]:
class FaceDetector:
    def __init__(self, 
                 modelFile="res10_300x300_ssd_iter_140000.caffemodel",
                 configFile = "deploy.prototxt",
                 max_interations = 300,
                 conf_threshold = 0.60,
                 normalized_dim = (32,32)):
        self.modelFile = modelFile
        self.configFile = configFile
        self.max_interations = max_interations
        self.conf_threshold = conf_threshold
        self.normalized_dim = normalized_dim
        self.net = cv2.dnn.readNetFromCaffe(self.configFile, self.modelFile)
        
    def extract_random_faces(self, filename, num_faces):
        captured_faces = []
        iterations = 0
        v_cap = cv2.VideoCapture(filename)
        v_length = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        while len(captured_faces) < num_faces and iterations < self.max_interations:
            iterations += 1
            v_cap.set(1, np.random.randint(v_length)-1)
            
            ret, img = v_cap.read()
        
            if ret == True:
                (h, w) = img.shape[:2]
                blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (103.93, 116.77, 123.68))
                self.net.setInput(blob)
                detections = self.net.forward()
                for i in range(detections.shape[2]):
                    confidence = detections[0, 0, i, 2]            
                    if confidence > self.conf_threshold:
                        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                        (x1, y1, x2, y2) = box.astype("int")
                        face=img[y1:y2, x1:x2]
                        
                        # normlize
                        face = cv2.resize(face, self.normalized_dim)
                        captured_faces.append(face)
        
        # When everything done, release the video capture and video write objects
        v_cap.release()
    
        return captured_faces

In [7]:
path = '../videos/dfdc_train_part_14'
modelFile = './saved_models/DDC_ResNet29v2_model.029.h5'

In [8]:
fd = FaceDetector(max_interations = 60)

In [9]:
model = load_model(modelFile)

In [11]:
df = pd.read_json(path + '/metadata.json')
df = df.T

In [21]:
df['predict'] = 0.0

In [22]:
df.sample(n)

,label,original,split,predict
ffscvgescz.mp4,FAKE,iezlgezozt.mp4,train,0.0
rtnhyhrxny.mp4,FAKE,gdavsauheg.mp4,train,0.0
nrpasaonsc.mp4,FAKE,msegqgxgbb.mp4,train,0.0
sqspjnqhun.mp4,FAKE,kxfuajzzdo.mp4,train,0.0
tpqdixzqqk.mp4,REAL,NaN,train,0.0
pxhzhjufts.mp4,REAL,NaN,train,0.0
vydaidnycn.mp4,FAKE,xhsijmkono.mp4,train,0.0
iqmvrahukb.mp4,FAKE,zyicfdblvi.mp4,train,0.0
wkfjsqzatw.mp4,REAL,NaN,train,0.0
pehzjkpbrd.mp4,FAKE,kyiivwctwe.mp4,train,0.0


In [23]:
nf = 20
nf_min = 5
n = 10
for index, row in tqdm(df.query('label == "REAL"').sample(n).iterrows(), total=n):
    faces = fd.extract_random_faces(path + '/' + index, nf)
    
    if len(faces) >= nf_min:
        faces = np.append(np.empty(shape=(0,32,32,3), dtype=np.int8), 
                      faces, 
                      axis=0)

        faces = faces.astype('float32') / 255
        faces_mean = np.mean(faces, axis=0)
        faces -= faces_mean
        predictions = model.predict(faces)
        predict = np.mean(predictions[:,0])
    else:
        predict = .5
    
    df.at[index, 'predict'] = predict
    print(index,row.label,predict)

 10%|█         | 1/10 [00:05<00:47,  5.24s/it]

ksdohprrko.mp4 REAL 0.99737227


 20%|██        | 2/10 [00:13<00:48,  6.03s/it]

kyiivwctwe.mp4 REAL 0.9699591


 30%|███       | 3/10 [00:18<00:40,  5.79s/it]

...

wtaghbgiln.mp4 REAL 0.9899476


 90%|█████████ | 9/10 [01:34<00:12, 12.43s/it]

pxhzhjufts.mp4 REAL 0.9680869


100%|██████████| 10/10 [01:39<00:00, 10.35s/it]

mbgiqctjzk.mp4 REAL 0.8995999


In [24]:
df.query('predict > 0')

,label,original,split,predict
xdxfmvnghz.mp4,REAL,NaN,train,0.500000
ltfagtaafd.mp4,REAL,NaN,train,0.998234
ksdohprrko.mp4,REAL,NaN,train,0.997372
kyiivwctwe.mp4,REAL,NaN,train,0.969959
pxhzhjufts.mp4,REAL,NaN,train,0.968087
wpdrxrheoy.mp4,REAL,NaN,train,0.500000
mbgiqctjzk.mp4,REAL,NaN,train,0.899600
bzqsxmvxny.mp4,REAL,NaN,train,0.905826
hwofkxxmyw.mp4,REAL,NaN,train,0.916553
wtaghbgiln.mp4,REAL,NaN,train,0.989948
